In [16]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
import os
from dotenv import load_dotenv
from sec_api import PdfGeneratorApi
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import PyPDFLoader,WebBaseLoader
import json
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



URL_PATHS =["https://apnews.com/article/bank-stress-tests-federal-reserve-private-credit-jpmorgan-citigroup-bd4c6049c0f060a6e43ec3aa229c22af","https://fortune.com/2025/06/28/bank-stress-test-results-federal-reserve-dividends-stock-buybacks/"]










In [17]:
loader = WebBaseLoader(URL_PATHS)
documents = loader.load()

print(f'Length of documents: {len(documents)}')

Length of documents: 2


In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=0)
splits = text_splitter.split_documents(documents)


print(f'Length of documents: {len(documents)} length of splits: {len(splits)}')

Length of documents: 2 length of splits: 43


In [19]:
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits,embedding=embeddings)

llm = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")









In [20]:
#summarize

chain = load_summarize_chain(llm=llm,chain_type="stuff")

chain.run(documents)





"Major banks have passed the Federal Reserve's stress tests, allowing them to issue dividends and buy back shares. The tests were less rigorous this year due to a weaker global economy, resulting in less severe hypothetical scenarios. The banks were found to be well-capitalized and resilient to potential economic downturns. The tests did not heavily assess exposure to private equity or private credit assets, which are growing concerns in the financial system. Overall, the banks are in a strong position to withstand financial stress."

In [21]:
question = "how were the stress tests less severe compared to previous years?"

retriever = MultiQueryRetriever.from_llm(retriever=vectordb.as_retriever(),llm=llm)
unique_docs = retriever.get_relevant_documents(query=question)
print(f'Length of unique documents: {unique_docs}')

Length of unique documents: [Document(metadata={'title': 'Easier bank stress tests clear the way for dividends and buybacks | Fortune', 'source': 'https://fortune.com/2025/06/28/bank-stress-test-results-federal-reserve-dividends-stock-buybacks/', 'description': 'All 22 banks tested would have remained solvent, the Fed said, despite absorbing roughly $550 billion in theoretical losses.', 'language': 'en'}, page_content='All of these less harmful, but simulated, drops mean there would be less damage to these banks’ balance sheets and less risk of these banks of potentially failing. Since the banks passed the 2024 tests, it was expected that the banks would pass the 2025 tests.\n\n\n\n“Large banks remain well capitalized and resilient to a range of severe outcomes,” said Michelle Bowman, the bank’s vice chair for supervision, in a statement. An appointee of President Trump, Bowman became the Fed’s vice chair of supervision earlier this month.\n\n\n\nThe Fed said it went with a less vigoro

In [22]:
prompt_template = """
provide a answer at the end based on the following pieces of context.
IF you do not know the answer, simply say you do not know. Do not make anything up.


{context}

Answer:"""



PROMPT = PromptTemplate(template=prompt_template,input_variables=['context','question'])

llm.predict(PROMPT.format_prompt(context=unique_docs,question=question).text)

'Based on the provided context, the major banks have passed a less vigorous stress test from the Federal Reserve in 2025. This indicates that the banks are well capitalized and resilient to severe outcomes, allowing them to withstand a hypothetical severe economic downturn. The stress tests were less intense due to the weakened global economy and previous volatility in test results. The banks were not heavily tested on their exposure to private equity assets or private credit, which are growing alarmingly quickly and could pose systemic risks to the financial system.'